<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
import __init__
from src.utilities import train_test_validation_splits, \
                            prepare_train_test_val_seqs_by_batches, \
                            tensor2dataloader, load_full_meltome_FLIP_db, \
                            seek_UniprotID_association_Meltome_prots,\
                            download_UniprotID_Alphafold_Structures,\
                            handle_nans,\
                            load_fasta_to_df
                            
import pandas as pd
import numpy as np
    
path_meltome = '../datasets/FLIP/splits/meltome/splits/mixed_split.csv'
complete_meltome_db = '../datasets/FLIP/splits/meltome/full_dataset_sequences.fasta'
Gustav_preprocesed_meltome_db = '../datasets/dataset_Gustav_repro/meltome_preprocessed.tsv'

full_meltome_db = load_full_meltome_FLIP_db(complete_meltome_db)

#splits_meltome = pd.read_csv(path_meltome, sep=',')
#filtered_set_gustav = pd.read_csv(Gustav_preprocesed_meltome_db, sep='\t')

# Absolute
target_cols = ['tm']
num_targets = len(target_cols)

# Load data
df = pd.read_csv('../datasets/dataset_Gustav_repro/meltome_preprocessed.tsv', sep='\t', index_col = 'key')

# Remove nan from df
df = handle_nans(df, target_columns = ['tm', 'sequence'], method='remove')

# delete duplicate cols
df = df.drop('sequence', 1)

# Load train, test, valid fasta
df_train = load_fasta_to_df('../datasets/dataset_Gustav_repro/meltome_train.faa')
df_valid = load_fasta_to_df('../datasets/dataset_Gustav_repro/meltome_valid.faa')
df_test = load_fasta_to_df('../datasets/dataset_Gustav_repro/meltome_test.faa')

# Merge
train = df_train.join(df, how = 'left').reset_index().rename(columns = {'tm':'target'})
val = df_valid.join(df, how = 'left').reset_index().rename(columns = {'tm':'target'})
test = df_test.join(df, how = 'left').reset_index().rename(columns = {'tm':'target'})






#ff=full_meltome_db[full_meltome_db['sequence'].isin(train['sequence'])]

0 observations were removed which had one or more unknown Tm
Final number of proteins: 25232


/tmp/ipykernel_2060334/29308749.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('sequence', 1)


In [2]:
from tqdm.auto import tqdm
import torch, os
from src.ESM2embeddings import ESM2embeddings
from src.utilities import train_test_validation_splits, prepare_train_test_val_seqs_by_batches, tensor2dataloader
from src.NeuralArchitectures import regressionHead
from src.Trainer import Trainer

structuredir = '../datasets/Structures/'
meltome_struct_dir_labels = '../datasets/'

In [3]:

import torch
import torch_geometric
import torch_sparse
from torch_geometric.nn import MessagePassing

import esm


pt_batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ESM2 = ESM2embeddings(type_embedding = "facebook/esm2_t33_650M_UR50D", device = device, type_tool='FacebookESM2')

dir_pre_generated_embeddings = '../prepro_embeddings/'

path_train= dir_pre_generated_embeddings+'train_set_embeddings_ESM2_Gustav_data.pth'
path_test= dir_pre_generated_embeddings+'test_set_embeddings_ESM2_Gustav_data.pth'
path_val= dir_pre_generated_embeddings+'val_set_embeddings_ESM2_Gustav_data.pth'



def embedding_builder_ESM2(ESM2, device, train, test, val, path_train, path_test, path_val):
    if os.path.isfile(path_train)==False and os.path.isfile(path_test)==False and os.path.isfile(path_val)==False:

        """batch_train, batch_test, batch_val = prepare_train_test_val_seqs_by_batches(train.sequence.to_list(), 
                                                                                    test.sequence.to_list(), 
                                                                                    val.sequence.to_list(), 
                                                                                    batch_size=2)"""
        print ("Creating Embeddings...")   
        train_embeddings = ESM2.esm2embedding(train, device, layer_index=33) #esm2embedding(train, model, batch_converter, device, layer_index=33)#BERT.embed_dataset(batch_train)
        torch.save(train_embeddings, path_train)
        test_embeddings = ESM2.esm2embedding(test, device, layer_index=33) #esm2embedding(test, model, batch_converter, device, layer_index=33)
        torch.save(test_embeddings, path_test)
        val_embeddings = ESM2.esm2embedding(val, device, layer_index=33) #esm2embedding(val, model, batch_converter, device, layer_index=33)
        torch.save(val_embeddings, path_val)
        print ("Loaded")
    else:
        print ("Loading Premade Embeddings...")
        train_embeddings = torch.load(path_train)
        test_embeddings = torch.load(path_test)
        val_embeddings = torch.load(path_val)
        print ("Loaded")
    return train_embeddings, test_embeddings, val_embeddings






In [4]:
train_embeddings, test_embeddings, val_embeddings = embedding_builder_ESM2(ESM2, device, train, test, val,  path_train, path_test, path_val)

train_loader = tensor2dataloader(train_embeddings, torch.from_numpy( train.target.to_numpy() ), batch_size=50)
test_loader = tensor2dataloader(test_embeddings, torch.from_numpy( test.target.to_numpy() ), batch_size=50)
val_loader = tensor2dataloader(val_embeddings, torch.from_numpy( val.target.to_numpy() ), batch_size=50)

Creating Embeddings...
1318 sequences were too long and have been truncated to 1000 AA


  0%|          | 0/1065 [00:00<?, ?it/s]

351 sequences were too long and have been truncated to 1000 AA


  0%|          | 0/320 [00:00<?, ?it/s]

208 sequences were too long and have been truncated to 1000 AA


  0%|          | 0/157 [00:00<?, ?it/s]

Loaded


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
model = regressionHead(train_embeddings.shape[1::])
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
loss= torch.nn.MSELoss()
epochs = 200
model, val_loss = Trainer.train_LLMRegresor(train_loader, val_loader, model, device, loss, opt, epochs)

TypeError: train_LLMRegresor() missing 1 required positional argument: 'epoch_num'

In [ ]:
test_set, test_labels = test_loader.dataset.tensors
loss_test, outcome = Trainer.test_model(model, test_set, test_labels, loss, device)
from src.utilities import plot_results

plot_results( outcome, test_labels) 